In [167]:
import pandas as pd
import datetime
import numpy as np
import pymongo
import json


concats = pd.DataFrame()
for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("data/"+str(i)+".csv")], sort = False,ignore_index=True)

In [78]:
concats = concats[pd.notnull(concats['FTR'])]
concats = concats.drop(columns = concats.columns[22:])
concats['ID_FTR'] = concats.FTR.astype('category').cat.codes

In [79]:
def stringToDate(string):
    string = string.split("/")
    if(len(string[2]) == 4):
        string[2] = string[2][2:]
    date = string[0]+"/"+ string[1]+"/20"+string[2]
    date = datetime.datetime.strptime(date, '%d/%m/%Y')
    return date

In [80]:
def getLastMatchId(id_match, nb_earlier_match):
    dates = concats['Date'].apply(stringToDate)
    #homepart
    ids_HH = dates[(dates < dates[id_match]) & (concats["HomeTeam"] == concats["HomeTeam"][id_match])].index
    ids_HA = dates[(dates < dates[id_match]) & (concats["AwayTeam"] == concats["HomeTeam"][id_match])].index
    ids_home = np.array(ids_HH,ids_HA)
    if(ids_home.shape[0] < nb_earlier_match):
        return np.nan
    else:
        ids_home = ids_home[ids_home.argsort()[-nb_earlier_match:][::-1]]
    #awaypart
    
    ids_AH = dates[(dates < dates[id_match]) & (concats["HomeTeam"] == concats["AwayTeam"][id_match])].index
    ids_AA = dates[(dates < dates[id_match]) & (concats["AwayTeam"] == concats["AwayTeam"][id_match])].index
    ids_away = np.array(ids_AH,ids_AA)
    if(ids_away.shape[0] < nb_earlier_match):
        return np.nan
    else:
        ids_away = ids_away[ids_away.argsort()[-nb_earlier_match:][::-1]]
    return np.array([ids_away, ids_home])

In [81]:
ids = pd.Series(concats.index).apply(getLastMatchId, args=(3,))

In [82]:
ids = pd.DataFrame(ids)
ids = ids.reindex(concats.index)
ids = ids.dropna()

In [83]:
concats = concats.fillna(0.0)

In [84]:


concats['AAG'] = np.nan
concats['AAS'] = np.nan
concats['AASO'] = np.nan
concats['AAF'] = np.nan
concats['AAC'] = np.nan
concats['AAY'] = np.nan
concats['AAR'] = np.nan

concats['HAG'] = np.nan
concats['HAS'] = np.nan
concats['HASO'] = np.nan
concats['HAF'] = np.nan
concats['HAC'] = np.nan
concats['HAY'] = np.nan
concats['HAR'] = np.nan

for i in ids.index:
    #awaypart
    ids_away = ids[0][i][0]
    avg_goals = 0
    avg_shots = 0
    avg_shots_on = 0
    avg_fouls = 0
    avg_corners = 0
    avg_yellow = 0
    avg_red = 0
    for k in ids_away:
        if(concats.loc[i,"AwayTeam"] == concats.loc[k,"AwayTeam"]):
            avg_goals += concats.loc[k,"FTAG"]
            avg_shots += concats.loc[k,"AS"]
            avg_shots_on += concats.loc[k,"AST"]
            avg_fouls += concats.loc[k,"AF"]
            avg_corners = concats.loc[k,"AC"]
            avg_yellow += concats.loc[k,"AY"]
            avg_red += concats.loc[k,"AR"]
        else:
            avg_goals += concats.loc[k,"FTHG"]
            avg_shots += concats.loc[k,"HS"]
            avg_shots_on += concats.loc[k,"HST"]
            avg_fouls += concats.loc[k,"HF"]
            avg_corners = concats.loc[k,"HC"]
            avg_yellow += concats.loc[k,"HY"]
            avg_red += concats.loc[k,"HR"]
            
    concats.loc[i,'AAG'] = avg_goals
    concats.loc[i,'AAS'] = avg_shots
    concats.loc[i,'AASO'] = avg_shots_on
    concats.loc[i,'AAF'] = avg_fouls
    concats.loc[i,'AAC'] = avg_corners
    concats.loc[i,'AAY'] = avg_yellow
    concats.loc[i,'AAR'] = avg_red

            
    ids_home = ids[0][i][1]
    avg_goals = 0
    avg_shots = 0
    avg_shots_on = 0
    avg_fouls = 0
    avg_corners = 0
    avg_yellow = 0
    avg_red = 0
    for k in ids_home:
        if(concats.loc[i,"HomeTeam"] == concats.loc[k,"AwayTeam"]):
            avg_goals += concats.loc[k,"FTAG"]
            avg_shots += concats.loc[k,"AS"]
            avg_shots_on += concats.loc[k,"AST"]
            avg_fouls += concats.loc[k,"AF"]
            avg_corners = concats.loc[k,"AC"]
            avg_yellow += concats.loc[k,"AY"]
            avg_red += concats.loc[k,"AR"]
        else:
            avg_goals += concats.loc[k,"FTHG"]
            avg_shots += concats.loc[k,"HS"]
            avg_shots_on += concats.loc[k,"HST"]
            avg_fouls += concats.loc[k,"HF"]
            avg_corners = concats.loc[k,"HC"]
            avg_yellow += concats.loc[k,"HY"]
            avg_red += concats.loc[k,"HR"]
        
       
    
    concats.loc[i,'HAG'] = avg_goals
    concats.loc[i,'HAS'] = avg_shots
    concats.loc[i,'HASO'] = avg_shots_on
    concats.loc[i,'HAF'] = avg_fouls
    concats.loc[i,'HAC'] = avg_corners
    concats.loc[i,'HAY'] = avg_yellow
    concats.loc[i,'HAR'] = avg_red

In [85]:
concats = concats.drop([x for x in concats.index.values if x not in ids.index.values])

In [86]:
concats = concats.drop(columns = concats.columns[4:6])
concats = concats.drop(columns = concats.columns[5:20])

In [87]:
normalized_df=(concats[concats.columns[6:]]-concats[concats.columns[6:]].mean())/concats[concats.columns[6:]].std()

In [88]:
concats[concats.columns[6:]] = normalized_df

In [143]:
concats.columns[6:13]

Index(['AAG', 'AAS', 'AASO', 'AAF', 'AAC', 'AAY', 'AAR'], dtype='object')

In [144]:
concats.columns[13:]

Index(['HAG', 'HAS', 'HASO', 'HAF', 'HAC', 'HAY', 'HAR'], dtype='object')

In [159]:
ids_team = list()
names = concats["HomeTeam"].drop_duplicates()
for i in names:
    df = np.array(concats[concats["HomeTeam"] == i].index, concats[concats["AwayTeam"] == i].index)
    ids_team.append(max(df))

values = list()
for i in range(len(names)):
    if(concats.loc[ids_team[i], "HomeTeam"] == names.iloc[i]):
        values.append(concats.loc[ids_team[i], concats.columns[13:]].values.astype(float))
    else:
        values.append(concats.loc[ids_team[i], concats.columns[6:13]].values.astype(float))
        
team_data = pd.DataFrame(values)
team_data["TeamName"] =  names.values

In [162]:
concats

,Div,Date,HomeTeam,AwayTeam,FTR,ID_FTR,AAG,AAS,AASO,AAF,AAC,AAY,AAR,HAG,HAS,HASO,HAF,HAC,HAY,HAR
54,F1,20/09/14,Nantes,Nice,H,2,0.235243,0.266572,-0.346070,-0.780578,2.451612,-0.994774,-0.580106,-0.962820,0.382529,0.074035,0.052537,-0.837804,-1.454220,-0.576492
57,F1,21/09/14,Lille,Montpellier,D,1,-0.564010,-0.170714,-0.995201,0.056229,-0.109168,-0.994774,-0.580106,-0.165427,0.049468,0.074035,0.052537,0.978304,-1.454220,-0.576492
60,F1,23/09/14,Reims,Marseille,A,0,1.034495,2.015716,1.384945,1.032504,0.256657,-0.994774,-0.580106,-0.165427,-1.393795,-0.794851,-0.798350,-1.564248,-1.936414,-0.576492
62,F1,24/09/14,Bastia,Nantes,D,1,-0.564010,0.922501,0.086683,0.893036,-0.109168,-1.478684,-0.580106,0.233269,-1.060734,-0.577630,1.612498,-1.564248,0.956749,-0.576492
64,F1,24/09/14,Evian Thonon Gaillard,Lens,H,2,-0.164384,0.047929,0.086683,0.614100,0.622483,1.424779,-0.580106,-1.361516,-1.171754,-1.880959,0.619796,-0.111361,-0.007638,-0.576492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2062,F1,07/12/2019,Nice,Metz,H,2,-0.963637,-0.061392,0.086683,-0.362175,-0.474994,-0.510863,-0.580106,1.429358,1.159671,1.811808,0.903425,0.615083,-0.007638,-0.576492
2063,F1,07/12/2019,Rennes,Angers,H,2,-0.564010,-0.389356,-0.562447,0.753568,-0.109168,0.456958,1.142606,0.631965,1.048651,0.942921,1.187054,-0.474582,0.474556,-0.576492
2064,F1,07/12/2019,Strasbourg,Toulouse,H,2,0.235243,-0.389356,0.086683,1.032504,-1.206645,0.456958,-0.580106,-1.361516,-0.172572,-0.577630,0.619796,-0.474582,0.474556,1.180820
2065,F1,08/12/2019,Reims,St Etienne,H,2,0.634869,0.485215,0.086683,0.893036,-0.840820,0.940868,-0.580106,-0.165427,0.604570,0.942921,-1.223794,-0.111361,-0.489832,-0.576492


In [161]:
team_data.to_csv("teamfeatures.csv")

In [138]:
concats.to_csv("parsed_train_data.csv")

In [192]:
def pandas_to_mongo(df):
    client = pymongo.MongoClient("mongodb://plp_mongodb:27017")
    db = client['refined-database']
    df.to_json('yourjson.json')                               # saving to json file
    jdf = open('yourjson.json').read()                        # loading the json file 
    data = json.loads(jdf)   
    posts = db.posts
    entry_list = list()
    for i in data.keys():
        entry_list.append(data[i])
    return posts.insert_many(entry_list)
    

In [193]:
pandas_to_mongo(team_data)
pandas_to_mongo(concats)

ServerSelectionTimeoutError: plp_mongodb:27017: [Errno 11001] getaddrinfo failed